In [2]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model

# Step 1: Load the Data
# Define your folder structure
data_dir = '../Ch11/cats_dogs/data/'
classes = ['cat', 'dog']

# Load and preprocess audio data
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []
    
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                
                # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
                mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                print(mel_spectrogram)
                data.append(mel_spectrogram)
                labels.append(i)
    
    return np.array(data), np.array(labels)

# Split data into training and testing sets
data, labels = load_and_preprocess_data(data_dir, classes) 
labels = to_categorical(labels, num_classes=len(classes))  # Convert labels to one-hot encoding
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


tf.Tensor(
[[[9.2251692e+00]
  [8.7654047e+00]
  [5.3550358e+00]
  ...
  [4.9005561e+00]
  [1.1241081e+00]
  [1.8701798e-01]]

 [[3.3728545e+00]
  [2.7357867e+00]
  [3.0335884e+00]
  ...
  [4.3269920e+00]
  [4.2865324e-01]
  [1.9584283e-01]]

 [[4.3421733e-01]
  [6.6295671e-01]
  [8.1721562e-01]
  ...
  [6.2421232e-01]
  [3.7745619e-01]
  [1.7195760e-01]]

 ...

 [[2.8427830e-04]
  [7.9414138e-04]
  [7.3328952e-04]
  ...
  [1.3638537e-04]
  [5.3319196e-05]
  [2.2383840e-05]]

 [[2.2763317e-04]
  [9.9361874e-04]
  [6.1165879e-04]
  ...
  [1.4689827e-04]
  [6.4195330e-05]
  [2.8969265e-05]]

 [[2.7415724e-04]
  [3.8014076e-04]
  [5.0225371e-04]
  ...
  [1.5754586e-04]
  [4.3283118e-05]
  [3.0331552e-05]]], shape=(128, 128, 1), dtype=float32)
tf.Tensor(
[[[2.24795237e-01]
  [1.54492044e+00]
  [1.44163644e+00]
  ...
  [1.16919827e+00]
  [1.94097352e+00]
  [8.73569250e-01]]

 [[5.29488735e-02]
  [1.37460423e+00]
  [1.57330871e+00]
  ...
  [1.17651641e+00]
  [1.21822834e+00]
  [1.12519240e+0

In [3]:
# Create a neural network model
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)

In [4]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 63, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 57600)             0     

In [5]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
6/6 [==============================] - 1s 158ms/step - loss: 11.5592 - accuracy: 0.7560 - val_loss: 20.6948 - val_accuracy: 0.8095
Epoch 2/20
6/6 [==============================] - 1s 103ms/step - loss: 3.2953 - accuracy: 0.9226 - val_loss: 6.9770 - val_accuracy: 0.8571
Epoch 3/20
6/6 [==============================] - 1s 113ms/step - loss: 0.4104 - accuracy: 0.9464 - val_loss: 4.3294 - val_accuracy: 0.8810
Epoch 4/20
6/6 [==============================] - 1s 103ms/step - loss: 0.1566 - accuracy: 0.9643 - val_loss: 3.1347 - val_accuracy: 0.8571
Epoch 5/20
6/6 [==============================] - 1s 119ms/step - loss: 0.0711 - accuracy: 0.9821 - val_loss: 2.8072 - val_accuracy: 0.8571
Epoch 6/20
6/6 [==============================] - 1s 99ms/step - loss: 0.0258 - accuracy: 0.9881 - val_loss: 2.5662 - val_accuracy: 0.8571
Epoch 7/20
6/6 [==============================] - 1s 99ms/step - loss: 0.0165 - accuracy: 0.9881 - val_loss: 2.5503 - val_accuracy: 0.8571
Epoch 8/20
6/6 [====

In [6]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8333333134651184


In [7]:
# Save the model
model.save('audio_classification_model.h5')

/Users/sudachk/Library/Python/3.9/lib/python/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Load the saved model
model = load_model('audio_classification_model.h5')

# Define the target shape for input spectrograms
target_shape = (128, 128)

# Define your class labels
classes = ['cat', 'dog']

# Function to preprocess and classify an audio file
def test_audio(file_path, model):
    # Load and preprocess the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))
    
    # Make predictions
    predictions = model.predict(mel_spectrogram)
    
    # Get the class probabilities
    class_probabilities = predictions[0]
    
    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)
    
    return class_probabilities, predicted_class_index

# Test an audio file
test_audio_file = '/Users/sudachk/Downloads/cat-meow-14536.mp3'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# Display results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

# Calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio is classified as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

1/1 [==============================] - 0s 120ms/step
Class: cat, Probability: 0.9999
Class: dog, Probability: 0.0001
The audio is classified as: cat
Accuracy: 0.9999


Model training with Augmented data:

In [9]:
import numpy as np
def manipulate(data, noise_factor):
    noise = np.random.randn(len(data))
    augmented_data = data + noise_factor * noise
    # Cast back to same data type
    augmented_data = augmented_data.astype(type(data[0]))
    return augmented_data

In [10]:
# Sample audio data
sample_data = np.array([0.1, 0.2, 0.3, 0.4, 0.5])

# Sample noise factor
sample_noise_factor = 0.05

# Apply augmentation
augmented_data = manipulate(sample_data, sample_noise_factor)

# Print the original and augmented data
print("Original Data:", sample_data)
print("Augmented Data:", augmented_data)


Original Data: [0.1 0.2 0.3 0.4 0.5]
Augmented Data: [0.13721247 0.21537706 0.33392584 0.40497867 0.5613516 ]


In [11]:
import os
import librosa
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.image import resize
from tensorflow.keras.models import load_model

# Step 1: Load the Data
# Define your folder structure
data_dir = '../Ch11/cats_dogs/data/'
classes = ['cat', 'dog']

# Load and preprocess audio data
def load_and_preprocess_data(data_dir, classes, target_shape=(128, 128)):
    data = []
    labels = []
    noise_factor = 0.05
    for i, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for filename in os.listdir(class_dir):
            if filename.endswith('.wav'):
                file_path = os.path.join(class_dir, filename)
                audio_data, sample_rate = librosa.load(file_path, sr=None)
                
                # Apply noise manipulation
                noise = np.random.randn(len(audio_data))
                augmented_data = audio_data + noise_factor * noise
                augmented_data = augmented_data.astype(type(audio_data[0]))

                # Perform preprocessing (e.g., convert to Mel spectrogram and resize)
                mel_spectrogram = librosa.feature.melspectrogram(y=augmented_data, sr=sample_rate)
                mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
                print(mel_spectrogram)
                
                data.append(mel_spectrogram)
                labels.append(i)
    return np.array(data), np.array(labels)

# Split data into training and testing sets
data, labels = load_and_preprocess_data(data_dir, classes)
labels = to_categorical(labels, num_classes=len(classes))  # Convert labels to one-hot encoding
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


tf.Tensor(
[[[12.160459  ]
  [ 9.766357  ]
  [ 5.3640676 ]
  ...
  [ 5.472106  ]
  [ 1.5104339 ]
  [ 0.60053587]]

 [[ 4.6013565 ]
  [ 2.5425313 ]
  [ 2.3898187 ]
  ...
  [ 5.6031413 ]
  [ 0.55699915]
  [ 0.6524204 ]]

 [[ 0.64814234]
  [ 1.5246296 ]
  [ 0.9170475 ]
  ...
  [ 0.8145863 ]
  [ 0.19996405]
  [ 0.36269534]]

 ...

 [[ 0.11517583]
  [ 0.22874856]
  [ 0.1848305 ]
  ...
  [ 0.19321406]
  [ 0.22389244]
  [ 0.21374457]]

 [[ 0.12406603]
  [ 0.23672527]
  [ 0.27168527]
  ...
  [ 0.30036044]
  [ 0.2411545 ]
  [ 0.19205727]]

 [[ 0.18800804]
  [ 0.25240245]
  [ 0.20964015]
  ...
  [ 0.23276646]
  [ 0.23218691]
  [ 0.16924942]]], shape=(128, 128, 1), dtype=float32)
tf.Tensor(
[[[0.8042023 ]
  [1.6535404 ]
  [2.0678477 ]
  ...
  [1.6216135 ]
  [2.8975148 ]
  [0.8724515 ]]

 [[0.3919087 ]
  [2.2786267 ]
  [1.9924153 ]
  ...
  [1.3924111 ]
  [1.6617535 ]
  [0.735692  ]]

 [[0.25153574]
  [2.5142205 ]
  [1.9034742 ]
  ...
  [1.9691011 ]
  [1.957476  ]
  [1.0962824 ]]

 ...

 [[0.252749

In [12]:
# Create a neural network model
input_shape = X_train[0].shape
input_layer = Input(shape=input_shape)
x = Conv2D(32, (3, 3), activation='relu')(input_layer)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)
output_layer = Dense(len(classes), activation='softmax')(x)
model = Model(input_layer, output_layer)

In [13]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 1)]     0         
                                                                 
 conv2d_2 (Conv2D)           (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 63, 63, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 30, 30, 64)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 57600)             0   

In [14]:
# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/20
6/6 [==============================] - 1s 122ms/step - loss: 74.5323 - accuracy: 0.6310 - val_loss: 10.8078 - val_accuracy: 0.7143
Epoch 2/20
6/6 [==============================] - 1s 120ms/step - loss: 1.9975 - accuracy: 0.7321 - val_loss: 3.6466 - val_accuracy: 0.7857
Epoch 3/20
6/6 [==============================] - 1s 102ms/step - loss: 0.5280 - accuracy: 0.8750 - val_loss: 2.6801 - val_accuracy: 0.8571
Epoch 4/20
6/6 [==============================] - 1s 102ms/step - loss: 0.3107 - accuracy: 0.9107 - val_loss: 2.8107 - val_accuracy: 0.8571
Epoch 5/20
6/6 [==============================] - 1s 99ms/step - loss: 0.2841 - accuracy: 0.9405 - val_loss: 2.9601 - val_accuracy: 0.8571
Epoch 6/20
6/6 [==============================] - 1s 100ms/step - loss: 0.2765 - accuracy: 0.9405 - val_loss: 3.1073 - val_accuracy: 0.8810
Epoch 7/20
6/6 [==============================] - 1s 101ms/step - loss: 0.2684 - accuracy: 0.9405 - val_loss: 3.3045 - val_accuracy: 0.8810
Epoch 8/20
6/6 [===

In [15]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.8571428656578064


In [16]:
# Load the saved model
model = load_model('audio_classification_model.h5')

# Define the target shape for input spectrograms
target_shape = (128, 128)

# Define your class labels
classes = ['cat', 'dog']

# Function to preprocess and classify an audio file
def test_audio(file_path, model):
    # Load and preprocess the audio file
    audio_data, sample_rate = librosa.load(file_path, sr=None)
    mel_spectrogram = librosa.feature.melspectrogram(y=audio_data, sr=sample_rate)
    mel_spectrogram = resize(np.expand_dims(mel_spectrogram, axis=-1), target_shape)
    mel_spectrogram = tf.reshape(mel_spectrogram, (1,) + target_shape + (1,))
    
    # Make predictions
    predictions = model.predict(mel_spectrogram)
    
    # Get the class probabilities
    class_probabilities = predictions[0]
    
    # Get the predicted class index
    predicted_class_index = np.argmax(class_probabilities)
    
    return class_probabilities, predicted_class_index

# Test an audio file
test_audio_file = '/Users/sudachk/Downloads/cat-meow-14536.mp3'
class_probabilities, predicted_class_index = test_audio(test_audio_file, model)

# Display results for all classes
for i, class_label in enumerate(classes):
    probability = class_probabilities[i]
    print(f'Class: {class_label}, Probability: {probability:.4f}')

# Calculate and display the predicted class and accuracy
predicted_class = classes[predicted_class_index]
accuracy = class_probabilities[predicted_class_index]
print(f'The audio is labeled as: {predicted_class}')
print(f'Accuracy: {accuracy:.4f}')

1/1 [==============================] - 0s 83ms/step
Class: cat, Probability: 0.9999
Class: dog, Probability: 0.0001
The audio is labeled as: cat
Accuracy: 0.9999
